# Create DFG Awards from GEPRIS

Creates DFG awards from the GEPRIS (German Project Information System) database. ~144K projects.

**Prerequisites:**
- Run `scripts/local/dfg_to_s3.py` to download and upload the data first.
- Alternative: Download from [Kaggle](https://www.kaggle.com/datasets/vicdoroshenko/dfg-gepris-funding-database-snapshot) and upload manually.

**Data source:** https://gepris.dfg.de/gepris/  
**S3 location:** `s3a://openalex-ingest/awards/dfg/dfg_projects.parquet`

**DFG funder in OpenAlex:**
- funder_id: 4320320879
- display_name: "Deutsche Forschungsgemeinschaft"
- ror_id: "https://ror.org/018mejw64"
- doi: "10.13039/501100001659"

**Input columns from dfg_to_s3.py:**
- project_id -> funder_award_id
- title -> display_name
- description -> description
- amount (DOUBLE, in EUR)
- program_type -> funder_scheme
- area -> subject_area
- start_date (STRING, YYYY-MM-DD)
- end_date (STRING, YYYY-MM-DD)
- start_year (INT)
- end_year (INT)
- lead_inst -> institution name
- pis_json -> PI data as JSON array

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.dfg_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/dfg/dfg_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~144K)
SELECT COUNT(*) as total_projects FROM openalex.awards.dfg_raw;

In [ ]:
%sql
-- Check column names to verify schema
DESCRIBE openalex.awards.dfg_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.dfg_raw LIMIT 5;

## Step 2: Create DFG Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.dfg_awards
USING delta
AS
WITH
-- Get DFG funder from OpenAlex by explicit funder_id
dfg_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320320879  -- Deutsche Forschungsgemeinschaft
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder_id:project_id
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.project_id)))) % 9000000000 as id,

        -- Display name = project title
        g.title as display_name,

        -- Description
        g.description as description,

        -- Funder info
        f.funder_id,
        g.project_id as funder_award_id,

        -- Amount (already in EUR from dfg_to_s3.py)
        TRY_CAST(g.amount AS DOUBLE) as amount,
        'EUR' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - map DFG programmes to types
        CASE
            WHEN LOWER(g.program_type) LIKE '%emmy noether%' THEN 'fellowship'
            WHEN LOWER(g.program_type) LIKE '%heisenberg%' THEN 'fellowship'
            WHEN LOWER(g.program_type) LIKE '%walter benjamin%' THEN 'fellowship'
            WHEN LOWER(g.program_type) LIKE '%research fellowship%' THEN 'fellowship'
            WHEN LOWER(g.program_type) LIKE '%stipend%' THEN 'fellowship'
            WHEN LOWER(g.program_type) LIKE '%graduiertenkolleg%' THEN 'training'
            WHEN LOWER(g.program_type) LIKE '%research training%' THEN 'training'
            WHEN LOWER(g.program_type) LIKE '%collaborative research%' THEN 'research'
            WHEN LOWER(g.program_type) LIKE '%sonderforschungsbereich%' THEN 'research'
            WHEN LOWER(g.program_type) LIKE '%priority programme%' THEN 'research'
            WHEN LOWER(g.program_type) LIKE '%schwerpunktprogramm%' THEN 'research'
            WHEN LOWER(g.program_type) LIKE '%forschergruppe%' THEN 'research'
            WHEN LOWER(g.program_type) LIKE '%research unit%' THEN 'research'
            WHEN LOWER(g.program_type) LIKE '%cluster%' THEN 'research'
            WHEN LOWER(g.program_type) LIKE '%excellence%' THEN 'research'
            WHEN LOWER(g.program_type) LIKE '%infrastructure%' THEN 'infrastructure'
            WHEN LOWER(g.program_type) LIKE '%equipment%' THEN 'infrastructure'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = program_type
        g.program_type as funder_scheme,

        -- Provenance
        'gepris' as provenance,

        -- Dates (stored as strings in YYYY-MM-DD format)
        COALESCE(
            TRY_TO_DATE(g.start_date, 'yyyy-MM-dd'),
            TRY_TO_DATE(CONCAT(g.start_year, '-01-01'), 'yyyy-MM-dd')
        ) as start_date,
        COALESCE(
            TRY_TO_DATE(g.end_date, 'yyyy-MM-dd'),
            TRY_TO_DATE(CONCAT(g.end_year, '-12-31'), 'yyyy-MM-dd')
        ) as end_date,
        COALESCE(
            YEAR(TRY_TO_DATE(g.start_date, 'yyyy-MM-dd')),
            TRY_CAST(g.start_year AS INT)
        ) as start_year,
        COALESCE(
            YEAR(TRY_TO_DATE(g.end_date, 'yyyy-MM-dd')),
            TRY_CAST(g.end_year AS INT)
        ) as end_year,

        -- Lead investigator - institution from lead_inst
        -- Note: GEPRIS data has institution but not individual PI names
        CASE
            WHEN g.lead_inst IS NOT NULL AND TRIM(g.lead_inst) != '' THEN
                struct(
                    CAST(NULL AS STRING) as given_name,
                    CAST(NULL AS STRING) as family_name,
                    CAST(NULL AS STRING) as orcid,
                    CAST(NULL AS DATE) as role_start,
                    struct(
                        g.lead_inst as name,
                        'Germany' as country,
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not available in GEPRIS data)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL - GEPRIS project page
        CONCAT('https://gepris.dfg.de/gepris/projekt/', g.project_id) as landing_page_url,

        -- No DOI for DFG grants typically
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.project_id)))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.dfg_raw g
    CROSS JOIN dfg_funder f
    WHERE g.project_id IS NOT NULL
      AND TRIM(CAST(g.project_id AS STRING)) != ''
)

SELECT * FROM awards_transformed;

In [ ]:
%sql
-- Remove previous data for this source before inserting fresh data
DELETE FROM openalex.awards.openalex_awards_raw
WHERE provenance = 'gepris' AND priority = 18;

-- Insert into openalex_awards_raw with priority
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    18 as priority
FROM openalex.awards.dfg_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count (should be ~144K)
SELECT COUNT(*) as total_dfg_awards FROM openalex.awards.dfg_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    start_date,
    end_date,
    lead_investigator
FROM openalex.awards.dfg_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be DFG)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.dfg_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.dfg_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme distribution (top 20 program types)
SELECT funder_scheme, COUNT(*) as cnt
FROM openalex.awards.dfg_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_description,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(end_date) as has_end_date,
    COUNT(lead_investigator) as has_institution,
    ROUND(COUNT(lead_investigator) * 100.0 / COUNT(*), 1) as pct_with_institution,
    ROUND(COUNT(description) * 100.0 / COUNT(*), 1) as pct_with_description
FROM openalex.awards.dfg_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt
FROM openalex.awards.dfg_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 20;

In [ ]:
%sql
-- Check institutions (top 20)
SELECT 
    lead_investigator.affiliation.name as institution,
    COUNT(*) as grant_count
FROM openalex.awards.dfg_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY grant_count DESC
LIMIT 20;